最近接到一个项目，需要计算大约 20 万个事件的 CAR。这是一个 CPU 密集型的任务，并行计算可以大大提高效率。于是我又重新捡起了进程、线程这些概念😒 每次都是用的时候看看文档，是时候进行一个系统性的梳理了🤣

# 进程，线程与协程

进程是一个程序在一个数据集中的一次动态执行过程，可以简单的理解为“正在执行的程序”，它是 CPU 资源分配和调度的独立单位。在 Windows 中可以通过任务管理器查看当下进程运行情况，比如我们打开文件资源管理器，就开了一个文件资源管理器进程（箭头1）。任务管理器显示了这个进程占用的 CPU、内存、网络等资源。

![图片1：任务管理器中的进程](./img/task-manager.png){width=100%}

进程之间相互隔离，每个进程拥有自己独立的内存空间、数据栈以及其他资源。因此现代应用程序一般会开启多个进程，尤其是在复杂的、资源密集型的应用中（例如网页浏览器、电子邮件客户端、开发工具等），这样可以提高稳定性、安全性和性能，一个进程崩溃也不会影响其他进程。比如我们打开一个计算器，操作系统开启两个进程（上图箭头2）。其中，runtime broker 是 Windows 操作系统中的系统进程，用于管理 UWP 应用的权限和安全。

进程的局限是创建，撤销和切换的开销比较大。同时，由于进程之间相互独立，进程间的通信比较复杂。另外，多进程并行运行时系统开销比较大。
 
线程是进程之后发展出来的概念，线程也叫轻量级进程，它是一个基本的 CPU 执行单元，也是程序执行过程中的最小单元。如果说进程是一个任务，线程可以理解为子任务。线程由线程 ID，程序计数器，寄存器集合和堆栈共同组成。每个进程内都有一个主执行线程，它无需由用户创建，而是由系统自动创建。用户根据需要在应用程序内创建其他线程，多个线程并发的运行于同一个进程中，共享进程内存。在 Windows 任务管理器中可以点击 details 查看线程。比如我们刚刚开启的文件资源管理器进程中，有 223 个线程在并发运行（下图箭头处）。

![图片2：任务管理器中的线程](./img/threads.png){width=100%}

线程的优点是减少了程序并发执行时的开销，提高了系统的并发性能；缺点是线程没有自己的系统资源，只有运行时必不可少的资源，但同一进程内的各线程可以共享进程所拥有的系统资源。另外，多个线程并发的运行在一个进程中，一个线程的崩溃可能影响整个进程。

最后简单提一下协程。协程是一种用户态的轻量级线程，又称微线程，协程的调度完全由用户控制，人们常常将协程与子程序（函数）比较着理解。子程序调用总是一个入口，一次返回，一旦退出即完成了子程序的执行。协程与多线程相比，其优势在于协程的执行效率极高，因为是子程序切换不是线程切换，由程序自身控制，因此没有线程切换的开销。

# 多进程 VS 多线程

程序任务可以简单分为两类：CPU 密集型和 IO 密集型。CPU 密集型任务是指程序的主要工作是进行大量的计算，比如加密、解密、大数据处理、矩阵运算等等，这类任务会占用大量的 CPU 资源。IO 密集型任务是指程序的大部分时间都在等待外部输入/输出，比如文件读写、网络请求、数据库查询等等，而不是占用 CPU 进行计算。多进程和多线程都可以实现多任务，但两者有不同的适用场景。其原因在于 Python 中存在一个全局解释器锁（GIL），这把锁只允许解释器中一次运行一个线程，也就是说同一时间只能有一个线程执行 Python 代码。

CPU 密集型任务适合多进程。Python 中的 GIL 只允许一个线程执行 Python 字节码。因此，对于 CPU 密集型任务，线程即使是并发执行，也不能真正利用多核 CPU 来加速运算，因为 GIL 限制了同一时间只能有一个线程运行 Python 代码。而多进程可以绕过 GIL。因为每个 Python 进程都有自己的解释器和 GIL，多个进程可以在多核 CPU 上并行运行，并充分利用 CPU 资源，提高运算效率。

IO 密集型任务适合多线程。IO 操作比如等待数据从网络传输，会让线程进入“阻塞”状态。此时，Python 解释器会释放 GIL，允许其他线程运行。因此，即使有 GIL 的限制，IO 密集型任务的多线程仍然能够并行处理多个任务，因为大部分时间线程都在等待外部资源，而不是执行 Python 代码。此外，线程的创建和切换开销比较小，适合大量并发的 IO 操作，例如处理多个网络请求或文件读写。Python 可以在等待 IO 操作时切换到其他线程继续执行。

# Python 的多进程实现